In [ ]:
# Import necessary libraries
import pandas as pd
import os
from pycaret.regression import *

In [ ]:
# Initiate important variables
target_id = [
    'CHEMBL238', 
    'CHEMBL218', 
    'CHEMBL3650', 
    'CHEMBL220', 
    'CHEMBL248', 
]
fingerprint_list = [
    'ECFP6_2048',
    'mol2vec',
    'pubchem',
    'pairfps',
    'rdk']
model_id = 'lightgbm'

In [ ]:
# Create and record model metrics in a for loop
for x in target_id:
    for y in fingerprint_list:
        df = pd.read_csv(f'fingerprints\{x}_{y}.csv')
        reg = setup(df, target='pIC50', session_id=42, use_gpu=True, train_size=0.7, normalize=True, normalize_method='minmax', silent=True)
        model = create_model(model_id)
        path = f'models/{model_id}/{x}'
        isExist = os.path.exists(path)
        if not isExist:
        # Create a new directory because it does not exist 
            os.makedirs(path)
        os.chdir(f'models/{model_id}/{x}')
        predicted = predict_model(model)
        predicted_metrics = pull()
        predicted_metrics['CHEMBLID'] = x
        predicted_metrics['fingerprint'] = y
        predicted_metrics['algorithm'] = model_id
        predicted_metrics.to_csv(f'{x}_{y}_{model_id}_predicted_metrics.csv', index = False)
        if y == 'pubchem':
            interpret_model(model, save = True)
            interpret_model(model, plot = 'pfi', save = True)
        final = finalize_model(model)
        save_model(final, f'{x}_{y}_{model_id}_pipeline')

        os.chdir('..')
        os.chdir('..')
        os.chdir('..')